## Correcting local biases in sampling

In [1]:
import scipy
import numpy as np
import pandas as pd
import itertools as it

from math import sin
import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)

from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import scale

from scipy.stats.stats import pearsonr 

from scipy.stats import invgamma 
from scipy.stats import beta
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In our first post we examined how to use frequency vectors to generate haplotypes and populations. We then proceeded to generate a universe of frequency vectors, whose distance in feature space allowed us to chose the relative differentiation of the populations we would simulate.

What i didn't touch on in that post was the importance of sampling in principal component analysis. In the last section, i chose vectors close to one another, together with vectors far distant, in order to produce differentiated populations. If you tweeked the population sizes, you might have noticed that if some of the close together populations largely outweighed the rest, the distances to the more differentiated clusters would be reduced.

- see [McVean 2009](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1000686) for a study case.


## Index

### I. Generating Frequency Vectors

### II. A One-Shot example

#### a. Biased sampling

#### b. MS correction

#### c. Unbiased sampling

### III. Permutations.




### I. Generating Frequency Vectors.

We will start by generating a space of vectors. 

For each vector we will extract _L_ random samples from the **beta** distribution. We will repeat this process as we vary the mean and variance (parameters _a_ and _b_) of the distribution. The ranges along which to vary these, the number of steps and the number of vectors to generate from each combination of _a_ and _b_ can be specified at the beginning of the next block of code.

In [2]:
# Simulate frequency vectors. 
# We must first define the number of populations, the length of the haplotypes desired, and their respective population sizes
L= 150

import itertools as it
n= 200

# Vary a (beta distribution parameter).
a_range= np.linspace(1,2,20)
a_set= [i for i in a_range for _ in range(n)]

# vary b.
b_range= np.linspace(0.1,.4,20)
b_set= [i for i in b_range for _ in range(n)]

## length of haplotypes to extract.
L_set= [L] * n * 20


background_1= np.array([a_set,b_set,L_set]).T

vector_lib= []
for k in range(background_1.shape[0]):
    
    probs= beta.rvs(background_1[k,0], background_1[k,1], size=int(background_1[k,2]))
    probs[(probs > 1)]= 1
    
    
    vector_lib.append(probs)

vector_lib= np.array(vector_lib)

In [3]:
print('Number of frequency vectors of size {} generated: {}'.format(vector_lib.shape[1],vector_lib.shape[0]))


Number of frequency vectors of size 150 generated: 4000


Perform PCA on data set of frequency vectors created.

This allows us to:

i) verify the desired randomness of the process of generating vectors.
 
ii) select vectors based on how much they resemble each other. 

We have verified in another post how distance in the feature space created this way relates positively to genetic distance as measured by Fst (see **10. A link to Fsts**).


In [4]:
## PCA on vectors simulated
n_comp = 100

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(vector_lib)
features = pca.transform(vector_lib)# * pca.explained_variance_ratio_

print("; ".join(['PC{0}: {1}'.format(x+1,round(pca.explained_variance_ratio_[x],3)) for x in range(n_comp)]))
print('features shape: {}'.format(features.shape))

PC1: 0.017; PC2: 0.009; PC3: 0.009; PC4: 0.009; PC5: 0.009; PC6: 0.009; PC7: 0.009; PC8: 0.009; PC9: 0.009; PC10: 0.009; PC11: 0.009; PC12: 0.009; PC13: 0.009; PC14: 0.008; PC15: 0.008; PC16: 0.008; PC17: 0.008; PC18: 0.008; PC19: 0.008; PC20: 0.008; PC21: 0.008; PC22: 0.008; PC23: 0.008; PC24: 0.008; PC25: 0.008; PC26: 0.008; PC27: 0.008; PC28: 0.008; PC29: 0.008; PC30: 0.008; PC31: 0.008; PC32: 0.008; PC33: 0.008; PC34: 0.008; PC35: 0.008; PC36: 0.008; PC37: 0.008; PC38: 0.008; PC39: 0.007; PC40: 0.007; PC41: 0.007; PC42: 0.007; PC43: 0.007; PC44: 0.007; PC45: 0.007; PC46: 0.007; PC47: 0.007; PC48: 0.007; PC49: 0.007; PC50: 0.007; PC51: 0.007; PC52: 0.007; PC53: 0.007; PC54: 0.007; PC55: 0.007; PC56: 0.007; PC57: 0.007; PC58: 0.007; PC59: 0.007; PC60: 0.007; PC61: 0.007; PC62: 0.007; PC63: 0.007; PC64: 0.007; PC65: 0.007; PC66: 0.007; PC67: 0.007; PC68: 0.007; PC69: 0.007; PC70: 0.007; PC71: 0.007; PC72: 0.006; PC73: 0.006; PC74: 0.006; PC75: 0.006; PC76: 0.006; PC77: 0.006; PC78: 0.

In [5]:
Iter= 50
target= [0,1]
stairs= 4

MRCA= np.random.choice(range(vector_lib.shape[0]),1)
calypso= []
feat= []

for inter in range(stairs):
    Pair= np.random.choice(range(vector_lib.shape[0]),2,replace= False)
    Pair[1]= MRCA
    print(Pair)
    
    coords= features[Pair,:]
    
    vector2= coords[target[1]] - coords[target[0]]
    for angle in np.linspace(-20,20,Iter):
        new_guy = coords[target[0]] + [angle / 10 * x for x in vector2]
        
        feat.append(new_guy)
        
        new_guy= pca.inverse_transform(new_guy)
        new_guy[new_guy < 0]= 0
        new_guy[new_guy > 1]= 1
        
        calypso.append(new_guy)

features= np.array(feat)
vector_lib= np.array(calypso)

[2542 1491]
[  96 1491]
[2340 1491]
[2749 1491]


In [6]:
features.shape

(200, 100)

In [8]:
## Plot vector PCA
fig_data= [go.Scatter3d(
        x = features[:,0],
        y = features[:,1],
        z = features[:,2],
        type='scatter3d',
        mode= "markers",
        text= ['a: {}; b: {}, L: {}; index = {}'.format(background_1[k,0],background_1[k,1],background_1[k,2], k) for k in range(background_1.shape[0])],
        marker= {
        'line': {'width': 0},
        'size': 2,
        'symbol': 'circle',
      "opacity": .6
      }
    )]


layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=fig_data)
iplot(fig)


 **Fig. 1** PCA on frequency vectors generated from the beta distribution. Parameters a and b were made to vary between 
1-2 and .1-.4 respectively at steps of .1. For each combination of parameters 15 vectors were produced.


The point of this is to peruse frequency vector space, and chose populations. Because PCA distances equal correlation between observations, the closer the points the higher the Fst between the pops selected.

### II. One-Shot example.

#### II a. Biased sampling of frequency vectors

chose populations and bias sizes below.

Haplotypes for each population will be generated using the frequency vectors as the probabilities of alleles.

In [12]:
### Select frequency vectors and draw haplotypes.
## Pops selected by Indicies.
N_pops= 4

Pops= np.random.choice(vector_lib.shape[0],N_pops,replace= False)

## Population Sizes and labels
Sizes_bias= [130,80,300,35]
labels_bias= np.repeat(np.array([x for x in range(N_pops)]),Sizes_bias)

## Number of pops

data_ex= []

for k in range(N_pops):
    
    probs= vector_lib[Pops[k],:]
    probs= [[0,x][int(x >= 0)] for x in probs]
    probs= [[1,x][int(x <= 1)] for x in probs]
    
    m= Sizes_bias[k]
    Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]
    
    data_ex.extend(Haps)

data_ex= np.array(data_ex)
print(data_ex.shape)



(545, 150)


In [13]:
data_ex[:20,:20]

array([[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0,

We will also calculate genetic distances between pairs of individuals based on the haplotypes. Calculation will be simple, distance based (n_diffs / hap_length).

In [14]:
### Calculate individual pairwise distances for biased sampling.
def pairwise_gen(x,y):
    miss= 0
    same= 0
    if len(x) != len(y):
        return 'vector lengths differ'
    else:
        for n in range(len(x)):
            if x[n] == y[n]:
                same += 1
        return 1 - same / (len(x) - miss)

bias_gen_diffs= pairwise_distances(data_ex,metric= pairwise_gen)
bias_gen_diffs= np.array(bias_gen_diffs)

iugen= np.triu_indices(bias_gen_diffs.shape[0],1)
bias_gen_diffs= bias_gen_diffs[iugen]

From the frequency vectors selected we can also calculate pairwise Fst's.

These will be used later to compare to pairswise centroid distances in feature space.

In [15]:
print(vector_lib.shape)
Pops

(200, 150)


array([123,  23, 159,  90])

In [16]:
### Calculate pairwise Fst based on frequency vectors selected.
def return_fsts2(freq_array):
    pops= range(freq_array.shape[0])
    H= {pop: [1-(freq_array[pop,x]**2 + (1 - freq_array[pop,x])**2) for x in range(freq_array.shape[1])] for pop in range(freq_array.shape[0])}
    Store= []

    for comb in it.combinations(H.keys(),2):
        P= [sum([freq_array[x,i] for x in comb]) / len(comb) for i in range(freq_array.shape[1])]
        HT= [2 * P[x] * (1 - P[x]) for x in range(len(P))]
        per_locus_fst= [[(HT[x] - np.mean([H[p][x] for p in comb])) / HT[x],0][int(HT[x] == 0)] for x in range(len(P))]
        per_locus_fst= np.nan_to_num(per_locus_fst)
        Fst= np.mean(per_locus_fst)

        Store.append([comb,Fst])
    
    
    ### total fst:
    P= [sum([freq_array[x,i] for x in pops]) / len(pops) for i in range(freq_array.shape[1])]
    HT= [2 * P[x] * (1 - P[x]) for x in range(len(P))]
    FST= np.mean([(HT[x] - np.mean([H[p][x] for p in pops])) / HT[x] for x in range(len(P))])
    
    return pd.DataFrame(Store,columns= ['pops','fst'])


freqs_selected= vector_lib[Pops,:]
Pairwise= return_fsts2(freqs_selected)

fsts_compare = Pairwise.fst
Pairwise

c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars



,pops,fst
0,"(0, 1)",0.089435
1,"(0, 2)",0.170857
2,"(0, 3)",0.117429
3,"(1, 2)",0.157459
4,"(1, 3)",0.113856
5,"(2, 3)",0.236405


Finally we perform PCA on the haplotypes generated and keep the first 5 PCs. Notice that right now the multiplication by eigenvalues is commented out. 

To run this test with the multiplication by eigenvalues you can uncomment that (i'll write something simpler later on).

In [17]:
### PCA on haplotypes drawn.
n_comp = 5

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data_ex)

bias_features= pca.transform(data_ex)# * pca.explained_variance_ratio_

var_comps= pca.explained_variance_ratio_
print("; ".join(['PC{0}: {1}'.format(x+1,round(var_comps[x],3)) for x in range(n_comp)]))
print(bias_features.shape)

PC1: 0.231; PC2: 0.051; PC3: 0.031; PC4: 0.015; PC5: 0.014
(545, 5)


Estimating population centroids in feature space and plot the PCA on biased-haplotype data set

In [18]:
## Calculate centroids
bias_centroids= [np.mean(bias_features[[y for y in range(bias_features.shape[0]) if labels_bias[y] == z],:],axis= 0) for z in range(N_pops)]
bias_centroids= np.array(bias_centroids)

## plot
fig_data= [go.Scatter(
        x = bias_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],0],
        y = bias_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],1],       
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]


fig_data.append(
    go.Scatter(
        x= bias_centroids[:,0],
        y= bias_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)

layout = go.Layout(
    title= 'Biased sampling; eigenvalues factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comps[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comps[0],3))),
)


fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 2** PCA on haplotypes sampled unevenly from the frequency vectors selected above. 

Calculate Pairwise centroid distances in feature space (remember we kept 5 components), and individual pairwise distances in this space also.

In [19]:
## centroid distances global
iu1= np.triu_indices(N_pops,1)
bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
bias_pair_dist= bias_pair_dist[iu1]
#bias_pair_dist= scale(bias_pair_dist)

### centroid distances by PC:
dist_PC_bias= {}
for PC in range(bias_centroids.shape[1]):
    bias_PC_dist= pairwise_distances(bias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
    bias_PC_dist= bias_PC_dist[iu1]
    #bias_pair_dist= scale(bias_pair_dist)
    dist_PC_bias[PC]= bias_PC_dist
    
## Individual distances:
bias_feat_dist= pairwise_distances(bias_features, metric= 'euclidean')
bias_feat_dist= bias_feat_dist[iugen]
bias_feat_dist= scale(bias_feat_dist)


### Pearson's r between individual pairwise genetic and feature space distances.
bias_feat_Pearson= pearsonr(bias_feat_dist,bias_gen_diffs)
print('Our first result: Pearson r between individual feature space and genetic distances: {}'.format(bias_feat_Pearson[0]))


Our first result: Pearson r between individual feature space and genetic distances: 0.9303081652834867


In [20]:
bias_pair_dist

array([ 3.09178259,  4.55637591,  3.65263401,  4.34161377,  3.48108167,
        5.38727355])

I chose the first two populations to play the outliers, the rest to be a close pack. To two of these i gave population sizes of 300 and 180, six and 3.6 times the size of the largest outlying population. 

The distortion can be seen in that our outlying populations don't appear as far as we would have expected them to given their vectors alone. They tend to appear in the center because of their reduced impact on variance components.

As remarked by McVean, this can be a problem when deriving conclusions from relative distances in feature space.


#### II. b. MeanShift correction.

MeanShift allows us to identify clusters in feature space, we will resample those clusters equally, inverse transform their coordinates and perform the PCA anew. The actual data is transposed onto the resulting space.

In [21]:
def local_sampling_correct(data_now,n_comp):
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data_now)
    feats= pca.transform(data_now)
    
    N= 50
    bandwidth = estimate_bandwidth(feats, quantile=0.15)
    params = {'bandwidth': np.linspace(np.min(feats), np.max(feats),30)}
    grid = GridSearchCV(KernelDensity(algorithm = "ball_tree",breadth_first = False), params,verbose=0)
    
    ## perform MeanShift clustering.
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, cluster_all=False, min_bin_freq=5)
    ms.fit(feats)
    labels1 = ms.labels_
    label_select = {y:[x for x in range(len(labels1)) if labels1[x] == y] for y in sorted(list(set(labels1))) if y != -1}

    ## Extract the KDE of each cluster identified by MS.
    Proxy_data= []

    for lab in label_select.keys():
        if len(label_select[lab]) < 3:
            continue
            
        Quanted_set= feats[label_select[lab],:]
        grid.fit(Quanted_set)

        kde = grid.best_estimator_
        Extract= kde.sample(N)
        Return= pca.inverse_transform(Extract)
        
        #Return= data_now[np.random.choice(label_select[lab],N),:]
        Proxy_data.extend(Return)
    
    Proxy_data= np.array(Proxy_data)
    
    print([len(x) for x in label_select.values()])
    pca2 = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(Proxy_data)
    var_comp= pca2.explained_variance_ratio_
    
    New_features= pca2.transform(data_now)# * var_comp
    return New_features, var_comp


New_features,var_comp= local_sampling_correct(data_ex,5)

[273, 126, 78, 35]


Plotting our original samples onto our re-computed feature space:

In [22]:
corr_centroids= [np.mean(New_features[[y for y in range(New_features.shape[0]) if labels_bias[y] == z],:],axis= 0) for z in range(N_pops)]
corr_centroids= np.array(corr_centroids)

fig_data= [go.Scatter(
        x = New_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],0],
        y = New_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],1],
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]

fig_data.append(
    go.Scatter(
        x= corr_centroids[:,0],
        y= corr_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)


layout = go.Layout(
    title= 'Biased corrected, eigenvalues not factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comp[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comp[0],3))),
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 3** MS corrected PCA of unvenly sampled haplotypes. MS clustering was first applied following an initial PCA of the biased data set. The KDE of each cluster identified this way was used to resample equally from that distribution. 50 observations were resampled from the distribution of each cluster identified by MS. Each observation from the resulting data set was inverse transformed and a new PCA was conducted on this data. Finally, the original haplotypes were projected onto the new feature space.

Very different from the original PCA on the biased data set.

We now calculate the distances between centroids in this feature space and the pairwise individual distances, compare the latter to genetic distances.

In [23]:
corr_centroids.shape

(4, 5)

In [24]:
### Centroid distances global
iu1= np.triu_indices(N_pops,1)
corrected_pair_dist= pairwise_distances(corr_centroids,metric= 'euclidean')
corrected_pair_dist= corrected_pair_dist[iu1]
#corrected_pair_dist= scale(corrected_pair_dist)

### centroid distances by PC
dist_PC_corrected= {}
for PC in range(corr_centroids.shape[1]):
    corrected_PC_dist= pairwise_distances(corr_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
    corrected_PC_dist= corrected_PC_dist[iu1]
    #corrected_pair_dist= scale(corrected_pair_dist)
    dist_PC_corrected[PC]= corrected_PC_dist

## Individual distances:
corrected_feat_dist= pairwise_distances(New_features, metric= 'euclidean')
corrected_feat_dist= corrected_feat_dist[iugen]
corrected_feat_dist= scale(corrected_feat_dist)

corrected_gen_pearson= pearsonr(corrected_feat_dist,bias_gen_diffs)

print('Pearon r of individual genetic distances versus feature space distances following correction: {}'.format(round(corrected_gen_pearson[0],3)))


Pearon r of individual genetic distances versus feature space distances following correction: 0.93


In [25]:
corrected_pair_dist

array([ 3.09186111,  4.55640989,  3.6527494 ,  4.34160672,  3.4810844 ,
        5.38732639])

#### II. c. Even sampling.


We can compare this output with what we would have gotten from sampling equally across our selected vectors.

For this purpose we sample equally from the same frequency vectors as in the biased scenario and perform PCA on the resulting data set.

In [26]:
#### Selecting new, equal sample sizes but derive haplotypes from the same frequency vectors.

Sizes= [50,50,50,50]
labels= np.repeat(np.array([x for x in range(N_pops)]),Sizes)

data= []

for k in range(N_pops):
    
    probs= vector_lib[Pops[k],:]
    probs= [[0,x][int(x >= 0)] for x in probs]
    probs= [[1,x][int(x <= 1)] for x in probs]
    
    m= Sizes[k]
    Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]
    
    data.extend(Haps)

data= np.array(data)
#data= scale(data)

#### clalculate pairwise genetic distances
iugen_unbiased= np.triu_indices(data.shape[0],1)

unbias_gen_diffs= pairwise_distances(data,metric= pairwise_gen)
unbias_gen_diffs= np.array(unbias_gen_diffs)

unbias_gen_diffs= unbias_gen_diffs[iugen_unbiased]

### perform PCA

n_comp = 5

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data)

features= pca.transform(data)# * pca.explained_variance_ratio_

var_comps= pca.explained_variance_ratio_
print("; ".join(['PC{0}: {1}'.format(x+1,round(var_comps[x],3)) for x in range(n_comp)]))
print(features.shape)


#### Calculate centroids of labelled data in feature space.
unbias_centroids= [np.mean(features[[y for y in range(features.shape[0]) if labels[y] == z],:],axis= 0) for z in range(N_pops)]
unbias_centroids= np.array(unbias_centroids)


#### Plot projections + Centroids.

fig_data= [go.Scatter(
        x = features[[x for x in range(sum(Sizes)) if labels[x] == i],0],
        y = features[[x for x in range(sum(Sizes)) if labels[x] == i],1],
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]


fig_data.append(
    go.Scatter(
        x= unbias_centroids[:,0],
        y= unbias_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)

layout = go.Layout(
    title= 'Unbiased sampling; eigenvalues factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comps[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comps[0],3))),
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

PC1: 0.183; PC2: 0.076; PC3: 0.067; PC4: 0.017; PC5: 0.016
(200, 5)


**Fig. 4** PCA on evenly sampled haplotypes from the same frequency vectors as above. 50 haplotypes generated from each vector.

Calculate pairwise distances. compare to genetic distances.

In [27]:
### centroid distances global
unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
unbias_pair_dist= unbias_pair_dist[iu1]
#unbias_pair_dist= scale(unbias_pair_dist)

### centroid distances by PC
dist_PC_even= {}
for PC in range(unbias_centroids.shape[1]):
    unbias_PC_dist= pairwise_distances(unbias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
    unbias_PC_dist= unbias_PC_dist[iu1]
    #unbias_pair_dist= scale(unbias_pair_dist)
    dist_PC_even[PC]= unbias_PC_dist

## Individual distances:
unbiased_feat_dist= pairwise_distances(features, metric= 'euclidean')

unbiased_feat_dist= unbiased_feat_dist[iugen_unbiased]
unbiased_feat_dist= scale(unbiased_feat_dist)

unbiased_gen_pearson= pearsonr(unbiased_feat_dist,unbias_gen_diffs)

print('Pearson r on individual genetic and feature space distances in the unbiased sampling scenario: {}'.format(round(corrected_gen_pearson[0])))


Pearson r on individual genetic and feature space distances in the unbiased sampling scenario: 1.0


In [28]:
fig_data= [go.Scatter(
    x= dist_PC_even[PC],
    y= dist_PC_corrected[PC],
    mode= 'markers',
    marker= dict(
        color= PC,
        opacity= .6
    ),
    name= 'PC: {}, {}'.format(PC,round(pearsonr(dist_PC_even[PC],dist_PC_corrected[PC])[0],2))
    ) for PC in dist_PC_even.keys()
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 5** Principal-component-wise relation between pairwise centroid distances in Unbiased versus Corrected scenarios. Average distances were calculated between the projections of population specific haplotypes along specific PCs. Pearson's r was calculated using the function `pearsonr` of the python package `scipy.stats.stats`.

In [29]:
t= np.array([
    unbias_pair_dist,
    bias_pair_dist,
    corrected_pair_dist
]).T

fig_data= [go.Scatter(
    x= t[:,0],
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['bias','corrected'][i-1]
    ) for i in [1,2]
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 6** Relation between pairwise centroid distances in unbiased scenario versus biased and corrected scenarios. Distances were left unscaled.

In [30]:

fig_fsts= [go.Scatter(
    x= fsts_compare,
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i]
    ) for i in [0,1,2]
]

layout = go.Layout(
    title= 'PCA to genetic distances',
    yaxis=dict(
        title='centrois distances in feature space'),
    xaxis=dict(
        title='normalized Fst')
)

fig= go.Figure(data=fig_fsts, layout=layout)
iplot(fig)

**Fig. 7** Relationship between Fst and feature space distances in the three scenarios considered: unbiased, biased and MS corrected.


### II. Permutations.

We will now repeat this process sequentially, to get an idea of how much this method actually corrects distances between pops.

At each repetition we will choose a fixed number of frequency vectors from the _Vector Universe_ created at the top of this page. We then perform a biased and an unbiased sampling of each, and perform PCA on both. For each scenario we calculate the pairwise eucledian distances between the centroids of populations and normalize them. 

We then apply the MScorrection to the feature space of the biased scenario and recalculate pairwise centroid distances and normalize them.

This will allow us to compare the unbiased distances to biased and corrected distances. Hopefully, we will have reduced the distortion produced by the biases in sampling.


- You can choose to to multiply PCs of PCAs performed by their respective eigenvalues by typing `Eigen = True` below.

- You can choose to scale Haplotype matrices produced by feature by typing `Scale = True` below.

In [302]:
### Select pre and post processing measures. 
Eigen = False
Scale= False
Center= False

MixL= False # select if to mix N_Pops or not. 
MixP= False # select if to mix lengths or not. 
Pairs= False # select if comparing Pairs of distances or the distances themselves
Control_inc= True

length_haps= 150
length_range= [75,300]
length_step= 10

pop_max= 6 # Number of pops

n_comp= 5 # components to keep following PCA

Iter= 20 # repeats

N_sims= 100 # number of haplotypes to generate from each pop in the unbiased scenario.

#def controled_fsts(vector_lib,Eigen,length_haps,Scale,Center,N_pops,n_comp,Iter,N_sims,MixL,MixP,Pairs):

### Control set to include in the transformation:
control_vecs= np.random.choice(range(vector_lib.shape[0]),2)
control_labels= np.repeat([0,1],N_sims)
### Control set distances
control_even_distances= []
control_bias_distances= []

### store distances between centroids
biased_pairwise= []
unbiased_pairwise= []
corrected_pairwise= []

### store PC projection:
dist_PC_even= {x:[] for x in range(n_comp)}
dist_PC_bias= {x:[] for x in range(n_comp)}
dist_PC_corrected= {x:[] for x in range(n_comp)}

### store increemental PC distances
dist_increment_even= {x:[] for x in range(1,n_comp)}
dist_increment_bias= {x:[] for x in range(1,n_comp)}    

### store fsts
fst_store= []


### proceed.

for rep in range(Iter):
    
    if MixP:
        N_pops= np.random.choice(range(3,pop_max),1,replace= False)[0]
    else: 
        N_pops= pop_max
    
    ## Population Sizes and labels
    bias_scheme= np.random.choice(range(25,200),N_pops,replace= False)
    unbiased_sheme= np.repeat(N_sims,N_pops)

    bias_labels= np.repeat(np.array([x for x in range(N_pops)]),bias_scheme)
    unbias_labels= np.repeat(np.array([x for x in range(N_pops)]),unbiased_sheme)

    ### triangular matrices extract.
    iu1= np.triu_indices(N_pops,1) # for centroid comparison

    iu_unbias= np.triu_indices(sum(unbiased_sheme),1)
    iu_bias= np.triu_indices(sum(bias_scheme),1)
    
    iu_control= np.triu_indices(2,1)
    
    Pops= np.random.choice(vector_lib.shape[0],N_pops,replace= False)
    print('vectors selected: {}'.format(Pops))
    ########## FST

    freqs_selected= vector_lib[Pops,:length_haps]
    Pairwise= return_fsts2(freqs_selected)

    #fsts_compare = scale(Pairwise.fst)
    fsts_compare= Pairwise.fst
    if Pairs:
        t= fsts_compare
        fsts_compare= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]
    
    fst_store.extend(fsts_compare)
    #########################################################
    ########### PCA ####################################
    #########################################################
    ### control sample
    
    control_data= []

    for k in range(2):

        probs= vector_lib[control_vecs[k],:length_haps]
        probs[probs < 0] = 0
        probs[probs > 1] = 1
        m= unbiased_sheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]
        
        control_data.extend(Haps)

    control_data= np.array(control_data)

    #### generate data and perform PCA.
    data= []

    for k in range(N_pops):

        probs= vector_lib[Pops[k],:length_haps]
        probs[probs < 0] = 0
        probs[probs > 1] = 1
        m= unbiased_sheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]

        data.extend(Haps)

    data1= np.array(data)

    if Scale:
        data1= scale(data1)
    
    if Control_inc:
        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(np.vstack([control_data,data1]))
        control_unbias_feat= pca.transform(control_data)
    else:
        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data1)
    
    feat_unbias= pca.transform(data1)

    if Eigen:
        feat_unbias= feat_unbias * pca.explained_variance_ratio_

    ####### centroid comparison
    #### Controls
    if Control_inc:
        control_centroids= [np.mean(control_unbias_feat[[y for y in range(control_unbias_feat.shape[0]) if control_labels[y] == z],:],axis= 0) for z in range(2)]
        control_centroids= np.array(control_centroids)

        unbias_control_dist= pairwise_distances(control_centroids,metric= 'euclidean')
        unbias_control_dist= unbias_control_dist[iu_control]

        control_even_distances.extend(unbias_control_dist)

    ####
    unbias_centroids= [np.mean(feat_unbias[[y for y in range(feat_unbias.shape[0]) if unbias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    unbias_centroids= np.array(unbias_centroids)

    unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
    unbias_pair_dist= unbias_pair_dist[iu1]

    if Pairs:
        t= unbias_pair_dist
        unbias_pair_dist= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]

    #unbias_pair_dist= scale(unbias_pair_dist)
    unbiased_pairwise.extend(unbias_pair_dist)

    ## PC-wise centroid comparison
    for PC in range(unbias_centroids.shape[1]):
        unbias_PC_dist= pairwise_distances(unbias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
        unbias_PC_dist= unbias_PC_dist[iu1]
        if Pairs:
            t= unbias_PC_dist
            unbias_PC_dist= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]
        
        dist_PC_even[PC].extend(unbias_PC_dist)
        if  PC > 0:
            unbias_increment_dist= pairwise_distances(unbias_centroids[:,:PC],metric= 'euclidean')
            unbias_increment_dist= unbias_increment_dist[iu1]
            dist_increment_even[PC].extend(unbias_increment_dist)            

    #################################################
    ############## biased sample

    #### generate data and perform PCA
    data= []

    for k in range(N_pops):

        probs= vector_lib[Pops[k],:]

        m= bias_scheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]

        data.extend(Haps)

    data2= np.array(data)

    if Scale:
        data2= scale(data2)
    
    if Control_inc:
        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(np.vstack([control_data,data2]))
        control_bias_feat= pca.transform(control_data)
    else:
        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data2)
    
    feat_bias= pca.transform(data2)

    if Eigen:
        feat_bias= feat_bias * pca.explained_variance_ratio_

    #### Centroid distances
    #### Controls
    if Control_inc:
        control_centroids= [np.mean(control_bias_feat[[y for y in range(control_bias_feat.shape[0]) if control_labels[y] == z],:],axis= 0) for z in range(2)]
        control_centroids= np.array(control_centroids)

        bias_control_dist= pairwise_distances(control_centroids,metric= 'euclidean')
        bias_control_dist= bias_control_dist[iu_control]

        control_bias_distances.extend(bias_control_dist)
    
    bias_centroids= [np.mean(feat_bias[[y for y in range(feat_bias.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    bias_centroids= np.array(bias_centroids)

    bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
    bias_pair_dist= bias_pair_dist[iu1]
    #bias_pair_dist= scale(bias_pair_dist)
    if Pairs:
        t= bias_pair_dist
        bias_pair_dist= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]
    
    biased_pairwise.extend(bias_pair_dist)

    ### PC-wise centroid comparison
    for PC in range(bias_centroids.shape[1]):
        bias_PC_dist= pairwise_distances(bias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
        bias_PC_dist= bias_PC_dist[iu1]
        if Pairs:
            t= bias_PC_dist
            bias_PC_dist= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]
        #bias_PC_dist= scale(bias_PC_dist)
        dist_PC_bias[PC].extend(bias_PC_dist)
        if PC > 0:
            bias_increment_dist= pairwise_distances(bias_centroids[:,:PC],metric= 'euclidean')
            bias_increment_dist= bias_increment_dist[iu1]
            #bias_PC_dist= scale(bias_PC_dist)
            dist_increment_bias[PC].extend(bias_increment_dist)

    ###############################################################"
    ################## bias correct
    ### perform MS correction on biased samples
    feat_correct,var_comp= local_sampling_correct(data2,n_comp)

    ### centroid Distances
    centroids= [np.mean(feat_correct[[y for y in range(feat_correct.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    centroids= np.array(centroids)
    pair_dist= pairwise_distances(centroids,metric= 'euclidean')
    pair_dist= pair_dist[iu1]
    #pair_dist= scale(pair_dist)
    if Pairs:
        t= pair_dist
        pair_dist= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]

    corrected_pairwise.extend(pair_dist)

    ### PC-wise centroid comparison
    for PC in range(centroids.shape[1]):
        corrected_PC_dist= pairwise_distances(centroids[:,PC].reshape(-1,1),metric= 'euclidean')
        corrected_PC_dist= corrected_PC_dist[iu1]
        #corrected_PC_dist= scale(corrected_PC_dist)
        if Pairs:
            t= corrected_PC_dist
            corrected_PC_dist= [min([t[y] for y in z]) / max([t[y] for y in z]) for z in it.combinations(range(len(t)),2)]

        dist_PC_corrected[PC].extend(corrected_PC_dist)
    
    

t= np.array([
fsts_compare,
unbias_pair_dist,
bias_pair_dist,
pair_dist
]).T



vectors selected: [ 68 130 117  66   2  92 165  63 155]


c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars



[354, 303, 238, 136, 49]
vectors selected: [100 152 119   7  71  32 156 121]


c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars



[335, 311, 115, 98, 81]
vectors selected: [119  22  24  26 105 160]


...

[433, 299]
vectors selected: [170 146  21  70]


c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars

c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning:

invalid value encountered in double_scalars



[140, 96, 75, 42]
vectors selected: [ 83  97 127   6  61 175]


c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars



[257, 88, 86]


In [303]:
### Distribution of feature space distances between control populations for even and biased scenarios
from sklearn.neighbors import KernelDensity

###

control_library= [control_even_distances,control_bias_distances]

X_plot = np.linspace(min(control_bias_distances) - 2, max(control_bias_distances) + 2, 1000)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(np.array(control_bias_distances).reshape(-1,1))

log_dens = kde.score_samples(X_plot.reshape(-1,1))

fig_roost_dens= [go.Scatter(x=X_plot, y=np.exp(log_dens), 
                            mode='lines', fill='tozeroy', name= 'Biased senarios',
                            line=dict(color='blue', width=2))]
##
X_plot = np.linspace(min(control_even_distances) - 2, max(control_even_distances) + 2, 1000)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(np.array(control_even_distances).reshape(-1,1))

log_dens = kde.score_samples(X_plot.reshape(-1,1))

fig_roost_dens.append(go.Scatter(x=X_plot, y=np.exp(log_dens), 
                            mode='lines', fill='tozeroy', name= 'even scenarios (n= {})'.format(N_sims),
                            line=dict(color='red', width=2)))

##

layout= go.Layout(
    title= 'PCA distances between control populations across iterations.'
)

fig = go.Figure(data=fig_roost_dens, layout= layout)
iplot(fig)

**Fig. 8** distribution of distances between the projections of control populations across iterations for even and biased sampling scenarios. Control vectors were maintained across iterations, PCA was performed with control samples included. New samples were generated at each iteration.

In [304]:
include= [x for x in range(len(fst_store)) if fst_store[x] <= 1 and fst_store[x] >= 0]


fig_data= [go.Scatter(
    x= [fst_store[x] for x in include],
    y= [dist_increment_even[PC][x] for x in include],
    mode= 'markers',
    marker= dict(
        color= PC,
        opacity= .6
    ),
    name= 'PC: {}, {}'.format(PC,round(pearsonr(fst_store,dist_increment_even[PC])[0],3))
    ) for PC in dist_increment_even.keys()
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='Fsts')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 9** Correlation between genetic and feature space distances calculated with incrementing number of PCs. Pearson's *r* was calculated on the concatenated vector of distances calculated across permutations. Distances (genetic and euclidian) were not normalized.

In [307]:
fig_data= [go.Scatter(
    x= dist_PC_even[PC],
    y= dist_PC_corrected[PC],
    mode= 'markers',
    marker= dict(
        color= PC,
        opacity= .6
    ),
    name= 'PC: {}, {}'.format(PC + 1,round(pearsonr(dist_PC_even[PC],dist_PC_corrected[PC])[0],2))
    ) for PC in dist_PC_corrected.keys()
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 10** PC-wise relation between pairwise centroid distances in unbiased scenario versus biased and corrected scenarios across simulations. Distances were left unscaled.

In [308]:
t= np.array([
    unbiased_pairwise,
    biased_pairwise,
    corrected_pairwise
]).T


In [309]:
pearsons= [pearsonr(fst_store,t[:,x])[0] for x in range(t.shape[1])]

fig_data= [go.Scatter(
    x= [fst_store[x] for x in include],
    y= [t[x,i] for x in include],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i] + ' r: {}'.format(round(pearsons[i],3))
    ) for i in [0,1,2]
]

layout = go.Layout(
    title= 'PCA distances against fst across sampling scenarios; VarEXC; Unscaled; Npops= {},(mixed={})'.format(pop_max,str(MixP)),
    yaxis=dict(
        range= [0,7],
        title='feature space distances'),
    xaxis=dict(
        range= [0,.4],
        title='Fst')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 11** Relationship between Fst and feature space distances across simulations and for the three scenarios considered: unbiased sampling, biased sampling and MS corrected biased sampling. first 10 principal components considered together.

In [301]:
pearsons= [pearsonr(fst_store,t[:,x])[0] for x in range(t.shape[1])]

fig_data= [go.Scatter(
    x= t[:,0],
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i] + ' r: {}'.format(round(pearsons[i],3))
    ) for i in [1,2]
]

layout = go.Layout(
    title= 'Biased and MS corrected against unbiased distances; Npops= {}'.format(N_pops),
    yaxis=dict(
        range= [0,7],
        title='biased and corrected distances'),
    xaxis=dict(
        range= [0,7],
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 12** Pairwise centroid distances in biased versus unbiased scenarios. Vectors were concatenated across iterations.

### Haplotype length and population number.

We began by conducting a test of the impact PCA on the relative distances of a single population genetic structure. Because a single scenario is not able to provide us with a statistical significance of the relation between genetic and PCA feature space distances, we then iterated that initial test across as many genetic structures as possible. This allowed us to extract an estimate of that relationship. We chose Pearson's *r* because of its apparent linearity.

However, in order to be able to compare different iterations we were limited to using the same haplotype length and population number. Going further, it is of interest to the practicioner to understand how this relation actually holds up across haplotype lengths and number of population numbers.


For this purpose we will package the block of script that performed the iterations into a function that takes as argument the number of populations and haplotype lengths.

This function is the core of the command-line ready script for this purpose. This script will take as arguments the total raange of population numbers and haplotype lengths to explore, and allow the user to select wether he wants his data to be scaled prior to feature reduction or not.

A [new directory]() was created to hold the final version of that product.

In [21]:

def controled_fsts(vector_lib,Eigen,length_haps,Scale,N_pops,n_comp,Iter,N_sims):

    ## Population Sizes and labels
    bias_scheme= np.random.choice(range(25,200),N_pops,replace= False)
    unbiased_sheme= np.repeat(N_sims,N_pops)

    bias_labels= np.repeat(np.array([x for x in range(N_pops)]),bias_scheme)
    unbias_labels= np.repeat(np.array([x for x in range(N_pops)]),unbiased_sheme)

    ### store distances between centroids
    biased_pairwise= []
    unbiased_pairwise= []
    corrected_pairwise= []

    ### store fsts
    fst_store= []

    ### store Pearson's r comparing gen_diffs and feature space diffs across scenarios
    biased_pears= []
    corrected_pears= []
    unbiased_pears= []

    ### triangular matrices extract.
    iu1= np.triu_indices(N_pops,1) # for centroid comparison

    iu_unbias= np.triu_indices(sum(unbiased_sheme),1)
    iu_bias= np.triu_indices(sum(bias_scheme),1)

    ### proceed.

    for rep in range(Iter):
        Pops= np.random.choice(vector_lib.shape[0],N_pops,replace= False)
        print('vectors selected: {}'.format(Pops))
        ########## FST

        freqs_selected= vector_lib[Pops,:length_haps]
        Pairwise= return_fsts2(freqs_selected)

        #fsts_compare = scale(Pairwise.fst)
        fsts_compare= Pairwise.fst
        fst_store.extend(fsts_compare)
        #########################################################
        ########### PCA ####################################
        #########################################################
        ############# unbiased sample

        #### generate data and perform PCA.
        data= []

        for k in range(N_pops):

            probs= vector_lib[Pops[k],:length_haps]

            m= unbiased_sheme[k]
            Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]

            data.extend(Haps)

        data1= np.array(data)

        if Scale:
            data1= scale(data1)

        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data1)

        feat_unbias= pca.transform(data1)

        if Eigen:
            feat_unbias= feat_unbias * pca.explained_variance_ratio_

        ####### centroid comparison
        unbias_centroids= [np.mean(feat_unbias[[y for y in range(feat_unbias.shape[0]) if unbias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
        unbias_centroids= np.array(unbias_centroids)

        unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
        unbias_pair_dist= unbias_pair_dist[iu1]

        #unbias_pair_dist= scale(unbias_pair_dist)
        unbiased_pairwise.extend(unbias_pair_dist)

        ######## ind distances
        ### genetic data
        unbias_gen_diffs= pairwise_distances(data1,metric= pairwise_gen)
        unbias_gen_diffs= np.array(unbias_gen_diffs)
        unbias_gen_diffs= unbias_gen_diffs[iu_unbias]

        ## feature space
        unbiased_feat_dist= pairwise_distances(feat_unbias, metric= 'euclidean')
        unbiased_feat_dist= unbiased_feat_dist[iu_unbias]
        #unbiased_feat_dist= scale(unbiased_feat_dist)

        unbiased_gen_pearson= pearsonr(unbiased_feat_dist,unbias_gen_diffs)

        unbiased_pears.append(unbiased_gen_pearson[0])

        #################################################
        ############## biased sample

        #### generate data and perform PCA
        data= []

        for k in range(N_pops):

            probs= vector_lib[Pops[k],:]

            m= bias_scheme[k]
            Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]

            data.extend(Haps)

        data2= np.array(data)

        if Scale:
            data2= scale(data2)

        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data2)

        feat_bias= pca.transform(data2)

        if Eigen:
            feat_bias= feat_bias * pca.explained_variance_ratio_

        #### Centroid distances
        bias_centroids= [np.mean(feat_bias[[y for y in range(feat_bias.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
        bias_centroids= np.array(bias_centroids)

        bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
        bias_pair_dist= bias_pair_dist[iu1]
        #bias_pair_dist= scale(bias_pair_dist)
        biased_pairwise.extend(bias_pair_dist)

        ######## Ind distances
        ### genetic data
        bias_gen_diffs= pairwise_distances(data2,metric= pairwise_gen)
        bias_gen_diffs= np.array(bias_gen_diffs)
        bias_gen_diffs= bias_gen_diffs[iu_bias]

        ## feature space
        biased_feat_dist= pairwise_distances(feat_bias, metric= 'euclidean')
        biased_feat_dist= biased_feat_dist[iu_bias]
        #biased_feat_dist= scale(biased_feat_dist)

        biased_gen_pearson= pearsonr(biased_feat_dist,bias_gen_diffs)

        biased_pears.append(biased_gen_pearson[0])

        ###############################################################"
        ################## bias correct
        ### perform MS correction on biased samples
        feat_correct,var_comp= local_sampling_correct(data2,n_comp)

        ### centroid Distances
        centroids= [np.mean(feat_correct[[y for y in range(feat_correct.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
        centroids= np.array(centroids)
        pair_dist= pairwise_distances(centroids,metric= 'euclidean')
        pair_dist= pair_dist[iu1]
        #pair_dist= scale(pair_dist)
        corrected_pairwise.extend(pair_dist)

        ######## Ind distances

        ## feature space
        corrected_feat_dist= pairwise_distances(feat_correct, metric= 'euclidean')
        corrected_feat_dist= corrected_feat_dist[iu_bias]
        #corrected_feat_dist= scale(corrected_feat_dist)

        corr_gen_pearson= pearsonr(corrected_feat_dist,bias_gen_diffs)

        corrected_pears.append(corr_gen_pearson[0])


        t= np.array([
        fsts_compare,
        unbias_pair_dist,
        bias_pair_dist,
        pair_dist
        ]).T

    t= np.array([
        unbiased_pairwise,
        biased_pairwise,
        corrected_pairwise
    ]).T

    pearsons= [pearsonr(fst_store,t[:,x])[0] for x in range(t.shape[1])]

    return pearsons


